In [0]:
%tensorflow_version 1.x
import numpy as np
from keras.models import Sequential
import keras
import tensorflow as tf
from keras.layers import Dense, Convolution2D, SpatialDropout2D, MaxPooling2D,Input, Flatten, Activation, Dropout
from keras.layers.core import Lambda
from keras import backend as K
from keras.regularizers import l2
from keras.models import Model
import os

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

def predict_pool_with_uncertainty(f, x, n_iter=50):
    """Function generating and returning non-deterministic predictions using MC dropout
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        All MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    return result

def fixed_mask_forward_pass (model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks):
      """Makes model predictions with J dropout masks that are fixed across points to enable estimation of Var(p_{sample})
    Function is specific to the given Keras model.
    #Arguments
        model: keras model
        forward_pass_input: X_{sample}
        num_masks: J, the number of dropout masks being used in estimation of Var(p_{sample}) and calculation of the EI acquisition function
        dropout_prob: dropout probability 
        conv_masks: first set of dropout masks, applied after MaxPooling2D layer
        dense_masks: second set of dropout masks, applied after the first Dense layer
    #Returns
        MC dropout predictions across sample points enabling estimation of Var(p_{sample}), i.e. \hat{p}_{sample}
    """
    conv = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[4].output])

    dense_1 = K.function([model.layers[6].input, K.learning_phase()],
                   [model.layers[8].output])

    dense_2 = K.function([model.layers[10].input, K.learning_phase()],
                   [model.layers[11].output])
    num_points = len(forward_pass_input)
    forward_pass_input = np.expand_dims(forward_pass_input, axis=1)
    conv_output = np.array(conv((forward_pass_input, 1)))
    dense_1_input = apply_dropout_masks(conv_output, conv_masks)
    if num_points == 1:
        dense_1_input = np.expand_dims(dense_1_input, axis=1)
    dense_1_output = multi_mask_predict(dense_1, dense_1_input)
    dense_2_input = apply_dropout_masks(dense_1_output, dense_masks)
    if num_points == 1:
        dense_2_input = np.expand_dims(dense_2_input, axis=1)
    dense_2_output = multi_mask_predict(dense_2, dense_2_input)
    return np.squeeze(dense_2_output)

def initialize_model (train_data_indices):
  """Used to initialize a Keras model in each active learning iteration; enables model retraining from scratch in each iteration
  #Arguments:
    train_data_indices: indices of training data within X_train_All (training set size affects weight decay)
  #Returns:
    A new initialized Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9] = 0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  train_size = y_train.shape[0]
  Weight_Decay = 0.0005/train_size
  dropout_prob = 0.30
  batch_size=128
  nb_filters = 35
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 2
  model = Sequential()
  model.add(Convolution2D(nb_filters, nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, nb_conv, data_format="channels_first", strides=2))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))
  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  return model

def run_model (train_data_indices):
  """Trains a Keras model with the training points specified by train_data_indices and evaluates model on test data
  #Arguments:
      train_data_indices: indices of current training points within X_train_All
  #Returns:
      Test accuracy and Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9]=0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  model = initialize_model(train_data_indices)
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input, K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), model]


def create_dropout_masks(dim, drop_prob):
    """Creates dropout masks that are fixed across all model inputs for use in estimation of Var(Y_{sample})
    Note: this function is specific to keras models used in this experiment; number of dropout masks J is deduced from dim.
    #Arguments
        dim: the numpy array shape of values entering the generated dropout masks for all points across all J fixed dropout masks
        drop_prob: the dropout probability
    #Returns
        dropout masks used in estimation of Var(p_{sample})
    """
    if (len(dim)==4):
        return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3])), p=[drop_prob, 1-drop_prob])
    return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3], dim[4])), p=[drop_prob, 1-drop_prob])

def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
    return np.array(layer_output)

def ei_batch_acquisition_unweighted(univ_cov, num_pool_samples, num_training_samples, batch_size):
    """Given Var(p_{sample}), applies batch-mode EI active learning to query points
    #Arguments
        univ_cov: Var(Y_{sample})
        num_pool_samples: number of pool points in D_{sample}
        num_training_samples: number of training points in D_{sample}
        batch_size: number of queried points per batch
    #Returns
        the variance reductions for queried points and the indices of queried pool points as points are arranged in univ_covariance
    """
  univ_covariance = univ_cov
  #Now starting batch mode acquisition loop
  acq_ind = []
  acq_vals = []
  for acq_num in range(batch_size):
    all_acq_values = np.zeros(num_pool_samples)
    for new_pt_ind in range(num_pool_samples):
      if new_pt_ind in acq_ind:
        all_acq_values[new_pt_ind] = 0.0
      else:
        start_ind = 2*(num_training_samples+new_pt_ind) 
        new_pt_cov_inv = np.linalg.inv(univ_covariance[start_ind:start_ind+2, start_ind:start_ind+2]) 
        covariance_matrix = univ_covariance[:, start_ind:start_ind+2] 
        first_matrix = np.matmul(covariance_matrix, new_pt_cov_inv)
        trace_reduction = np.sum(np.multiply(first_matrix, covariance_matrix)) 
        all_acq_values[new_pt_ind] = trace_reduction
    sorted_top_ind = np.flip(np.argsort(all_acq_values))
    found_new_ind = False
    top_ind_ctr = -1
    while (found_new_ind == False):
       top_ind_ctr += 1
       new_top_ind = sorted_top_ind[top_ind_ctr]
       if new_top_ind not in acq_ind:
         acq_ind.append(new_top_ind)
         acq_vals.append(all_acq_values[new_top_ind])
         found_new_ind = True
    update_start_ind = 2*(num_training_samples+acq_ind[-1])
    top_cov_matrix = univ_covariance[:, update_start_ind:update_start_ind+2]
    top_pt_cov_inv = np.linalg.inv(univ_covariance[update_start_ind:update_start_ind+2, update_start_ind:update_start_ind+2]) 
    first_matrix = np.matmul(top_cov_matrix, top_pt_cov_inv)
    univ_covariance = univ_covariance - np.matmul(first_matrix, top_cov_matrix.T) + (delta_identity * np.identity(univ_covariance.shape[0])) 
  return [acq_vals, acq_ind]

def get_acquisition_fn (model, X_train_sample, X_cand, num_masks, tau_inverse, batch_size, dropout_prob):
    """Given sample points, generates J fixed-mask predictions across all sample points, calculates Var(p_{sample}), and runs batch-mode EI acquisition
    #Arguments
      model: keras model
      X_train_sample: numpy array of sample points that are in the training set
      X_cand: numpy array of sample points that are in the pool
      num_masks: J, the number of dropout masks being used in estimation of Var(p_{sample}) and calculation of the EI acquisition function
      tau_inverse: smoothing parameter ensuring Var(p_{new}) is invertible
      batch_size: number of queried points per batch
      dropout_prob: dropout probability 
    #Returns
      The indices of pool points queried by batch-mode EI as they are arranged in X_{cand}
    """
    forward_pass_input = np.concatenate((X_train_sample, X_cand))
    forward_pass_output = np.zeros((2, len(X_train_sample)+len(X_cand), num_masks))
    #Generating J fixed masks used in obtaining sample predictions and calculating Var(p_{sample})
    conv_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 35, 3, 3)), p=[dropout_prob, 1-dropout_prob])
    dense_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 1, 128)), p=[dropout_prob, 1-dropout_prob])
    forward_pass_output = fixed_mask_forward_pass(model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks).T
    reshaped_forward_pass_output = np.zeros((2*(len(X_train_sample)+len(X_cand)), num_masks))
    for sample_pt_ind in range(len(X_train_sample)+len(X_cand)):
      reshaped_forward_pass_output[2*sample_pt_ind,:] = forward_pass_output[0, sample_pt_ind, :]
      reshaped_forward_pass_output[2*sample_pt_ind+1,:] = forward_pass_output[1, sample_pt_ind, :]
    output_covariance = np.cov(reshaped_forward_pass_output)
    print('num training: ' + str(len(X_train_sample))+', num cand: ' + str(len(X_cand)))
    #Incorporating smoothing parameter
    final_output_covariance = output_covariance + (tau_inverse * np.identity(output_covariance.shape[0]))
    return ei_batch_acquisition_unweighted (final_output_covariance, len(X_cand), len(X_train_sample), batch_size) 

In [0]:
#Active learning settings/parameters
dropout_prob = 0.3
tau_inverse = 0.0000837
num_experiments = 3
acq_fn_dropout_iterations = 50
num_acquisitions = 1000
sample_size = 2500
num_masks = 50

batch_size = 1


In [0]:
#Loading data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
starting_ind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Scripts/"
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Results/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
			delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
			delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.concatenate((np.argwhere(y_train_All==7), np.argwhere(y_train_All==9))).flatten()
test_ind = np.concatenate((np.argwhere(y_test==7), np.argwhere(y_test==9))).flatten()
y_test = y_test[test_ind]
y_test[y_test==9] = 0
y_test[y_test==7]= 1
y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=2)
X_test = np.expand_dims(X_test[test_ind], axis=1)

#Iterating across active learning experiments, each of which starts with a different initial training set
#Note: code is currently set up to *resume* an active learning experiment
#To start an active learning experiment from scratch, several lines below that are commented out should be uncommented, and vice versa

for e in range(0,1):
  acc_file = "EI_BS"+str(batch_size)+"_TauInv"+str(tau_inverse)+"_Acc_Ind" + str(e+1) + ".npy"
  ind_file = "EI_BS"+str(batch_size)+"_TauInv"+str(tau_inverse)+"_Ind_Ind" + str(e+1) + ".npy"
  #exp_acc = []
  exp_acc = list(np.load(results_path+acc_file))
  #train_data_indices = list(np.load(starting_ind_path + 'trainindices' + str(e+1) + '.npy'))
  train_data_indices = list(np.load(results_path+ind_file))
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  [init_acc, model] = run_model(train_data_indices)
  #exp_acc.append(init_acc)
  num_acquisitions = num_acquisitions - batch_size * (len(exp_acc) - 1)

  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    #Selecting sample points
    sample_indices = np.random.choice(len(pool_indices)+len(train_data_indices), int(sample_size*1.2), replace=False)
    pool_sample_ratio = len(sample_indices[sample_indices >= len(train_data_indices)])/sample_size
    train_sample_size = len(sample_indices[sample_indices < len(train_data_indices)])
    sample_indices_train = (sample_indices[sample_indices < len(train_data_indices)])[0:int(train_sample_size//pool_sample_ratio)]
    sample_indices_pool = (sample_indices[sample_indices >= len(train_data_indices)])[0:sample_size]
    X_train_fn = X_train_All[train_data_indices][sample_indices_train]
    X_pool_fn = np.concatenate((X_train_All[train_data_indices], X_train_All[pool_indices]))[sample_indices_pool]
    #EI acquisition
    acq_fn_results = get_acquisition_fn(model, X_train_fn, X_pool_fn, acq_fn_dropout_iterations, tau_inverse, batch_size, dropout_prob)
    acq_ind_ind_ind = np.array(acq_fn_results[1])
    acq_ind_ind = np.array((sample_indices_pool)[acq_ind_ind_ind]) - len(train_data_indices)
    all_acq_ind = np.array(pool_indices)[acq_ind_ind]
    #Incorporating queried points into training set
    for acq_ind in all_acq_ind:
      train_data_indices.append(acq_ind)
      pool_indices.remove(acq_ind)
    #Training a new model from scratch, evaluating on test data, and saving the results
    [acc, model] = run_model(train_data_indices)
    exp_acc.append(acc)
    print('Experiment ' + str(e+1) +", Total training points: " + str(len(train_data_indices)) + ", exp acc: " + str(exp_acc))
    np.save(results_path+acc_file, np.array(exp_acc))
    np.save(results_path+ind_file, np.array(train_data_indices))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`



num training: 149, num cand: 2500


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 677, exp acc: [0.8438880706921944, 0.8443789887088856, 0.8944526264113893, 0.8355424644084438, 0.8998527245949927, 0.865979381443299, 0.8915071183112421, 0.9273441335297006, 0.865979381443299, 0.9185076092292587, 0.9342169857633775, 0.9533627884143349, 0.9459990181639666, 0.9405989199803633, 0.9440353460972017, 0.93519882179676, 0.9671084928816888, 0.9278350515463918, 0.9666175748649976, 0.9347079037800687, 0.9587628865979382, 0.9656357388316151, 0.9386352479135984, 0.958271968581247, 0.9533627884143349, 0.9563082965144821, 0.9705449189985272, 0.96759941089838, 0.9661266568483063, 0.9666175748649976, 0.9671084928816888, 0.9685812469317624, 0.9710358370152185, 0.9597447226313206, 0.9612174766813942, 0.976926853215513, 0.9661266568483063, 0.9690721649484536, 0.9754540991654393, 0.9759450171821306, 0.9504172803141875, 0.9783996072655866, 0.9754540991654393, 0.9720176730486009, 0.9754540991654393, 0.9739813451153657, 0.9759450171821306, 0.98183603338242

...

Experiment 1, Total training points: 778, exp acc: [0.8438880706921944, 0.8443789887088856, 0.8944526264113893, 0.8355424644084438, 0.8998527245949927, 0.865979381443299, 0.8915071183112421, 0.9273441335297006, 0.865979381443299, 0.9185076092292587, 0.9342169857633775, 0.9533627884143349, 0.9459990181639666, 0.9405989199803633, 0.9440353460972017, 0.93519882179676, 0.9671084928816888, 0.9278350515463918, 0.9666175748649976, 0.9347079037800687, 0.9587628865979382, 0.9656357388316151, 0.9386352479135984, 0.958271968581247, 0.9533627884143349, 0.9563082965144821, 0.9705449189985272, 0.96759941089838, 0.9661266568483063, 0.9666175748649976, 0.9671084928816888, 0.9685812469317624, 0.9710358370152185, 0.9597447226313206, 0.9612174766813942, 0.976926853215513, 0.9661266568483063, 0.9690721649484536, 0.9754540991654393, 0.9759450171821306, 0.9504172803141875, 0.9783996072655866, 0.9754540991654393, 0.9720176730486009, 0.9754540991654393, 0.9739813451153657, 0.9759450171821306, 0.98183603338242

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 779, exp acc: [0.8438880706921944, 0.8443789887088856, 0.8944526264113893, 0.8355424644084438, 0.8998527245949927, 0.865979381443299, 0.8915071183112421, 0.9273441335297006, 0.865979381443299, 0.9185076092292587, 0.9342169857633775, 0.9533627884143349, 0.9459990181639666, 0.9405989199803633, 0.9440353460972017, 0.93519882179676, 0.9671084928816888, 0.9278350515463918, 0.9666175748649976, 0.9347079037800687, 0.9587628865979382, 0.9656357388316151, 0.9386352479135984, 0.958271968581247, 0.9533627884143349, 0.9563082965144821, 0.9705449189985272, 0.96759941089838, 0.9661266568483063, 0.9666175748649976, 0.9671084928816888, 0.9685812469317624, 0.9710358370152185, 0.9597447226313206, 0.9612174766813942, 0.976926853215513, 0.9661266568483063, 0.9690721649484536, 0.9754540991654393, 0.9759450171821306, 0.9504172803141875, 0.9783996072655866, 0.9754540991654393, 0.9720176730486009, 0.9754540991654393, 0.9739813451153657, 0.9759450171821306, 0.98183603338242

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Experiment 1, Total training points: 780, exp acc: [0.8438880706921944, 0.8443789887088856, 0.8944526264113893, 0.8355424644084438, 0.8998527245949927, 0.865979381443299, 0.8915071183112421, 0.9273441335297006, 0.865979381443299, 0.9185076092292587, 0.9342169857633775, 0.9533627884143349, 0.9459990181639666, 0.9405989199803633, 0.9440353460972017, 0.93519882179676, 0.9671084928816888, 0.9278350515463918, 0.9666175748649976, 0.9347079037800687, 0.9587628865979382, 0.9656357388316151, 0.9386352479135984, 0.958271968581247, 0.9533627884143349, 0.9563082965144821, 0.9705449189985272, 0.96759941089838, 0.9661266568483063, 0.9666175748649976, 0.9671084928816888, 0.9685812469317624, 0.9710358370152185, 0.9597447226313206, 0.9612174766813942, 0.976926853215513, 0.9661266568483063, 0.9690721649484536, 0.9754540991654393, 0.9759450171821306, 0.9504172803141875, 0.9783996072655866, 0.9754540991654393, 0.9720176730486009, 0.9754540991654393, 0.9739813451153657, 0.9759450171821306, 0.98183603338242